<div align="right"><i>Peter Norvig<br>2012; updated August 2019</i></div>

# Weighing Twelve Balls on a Balance Scale

> *You are given twelve identical-looking balls and a two-sided scale. One of the balls is of a different weight, although you don't know whether it's lighter or heavier. How can you use just three weighings of the scale to determine not only what the different ball is, but also whether it's lighter or heavier?*

This is a traditional brain-teaser puzzle, meant to be solved with paper and pencil. 
But I want to solve not just this specific puzzle, but related puzzles where you can vary (a) the number of balls, (b) the number of weighings allowed, and (c) whether the odd ball might be heavier, lighter, or either, or neither. For that I'll ned a program. (I originally wrote this program in 2012, but am republishing it here in revised form because the problem was mentioned in the [538 Riddler](https://fivethirtyeight.com/features/which-billiard-ball-is-rigged/) for 16 August 2019. It also appeared in a [Numberplay](https://wordplay.blogs.nytimes.com/2014/07/21/12coin/) column in 2014, with coins instead of balls.)

# Design

Here are the concepts I'm dealing with:

- **balls**: In the general case I have N balls. I'll represent them with a list like `[1, 2, 3]` for N = 3.
- **oddballs**: Exactly one of the balls is **odd** in its weight. 
I'll represent the situation where ball N is heavier as +N, and where ball N is lighter as -N. (I'll represent the situation where no ball is odd with `0`; that's not needed for the puzzle stated above, but is a nice extension.) With N = 3, the set of all possible oddballs is `{+1, -1, +2, -2, +3, -3}`. 
- **puzzle**: I'll call `Puzzle(N, W, {-1, +1})` to create a puzzle with N balls, W weighings allowed, and the oddballs allowing for each ball to be lighter (-1) or heavier (+1). The third argument can be any subset of `{-1, 0, +1}`, where the `0` stands for "allow the possibility that no ball is odd; they all weigh the same." 
- **weighing**: I can weigh a collection of balls on the left versus a collection on the right, and the result will be that the left side is greater than, equal to, or less than the right in weight. I'll denote that with the call `weigh(L, R, oddball)`, which returns a string, `'gt'`, `'eq'`, or `'lt'`.
- **weight**: I'll arbitrarily say that a normal ball weighs 100, a lighter ball 99, and a heavier ball 101.
- **solution**: Given a puzzle, a solution is a **strategy tree** that can correctly discover the odd ball, whatever it is, in the allowable number of weighings.
- **strategy tree**: A tree with oddballs (integers) as leaf nodes, and interior nodes with 5 components:  `Tree(L, R, gt, eq, lt)` is a node where `L` and `R` are the collections of balls to be weighed, and `gt`, `eq`, and `lt` are subtrees for the three possible results of the weighing.
- **following a path in a tree**: I'll use `follow(tree, oddball)` to say *follow the path through the tree, doing each weighing under the assumption of the given oddball, and return the leaf node reached by the path&mdash;the oddball that the tree predicts.* Note that the function `follow` gets to see what the oddball is, but the `tree` never gets direct access to that; the tree has to figure it out by doing weighings.
- **valid tree**: a tree is a valid solution to a puzzle if no branch uses more than the allowable number of weighings, and if, for every possible oddball in the puzzle, following the path through the tree correctly returns that oddball as the answer. I'll use `valid(tree, puzzle)` for this.


# Implementation

Let's start implementing the concepts:

In [1]:
from collections import namedtuple
import random

#### Types

Ball = Oddball = int

class Puzzle:
    "Represent a specific ball-weighing puzzle."
    def __init__(self, N=12, weighings=3, oddities={-1, +1}):
        self.weighings = weighings
        self.balls     = list(range(1, N + 1))
        self.oddballs  = {b * o for b in self.balls for o in oddities}
        
Tree = namedtuple('Tree', 'L, R, gt, eq, lt')

#### Functions
    
def weigh(L, R, oddball) -> str:
    "Weigh balls L against balls R, given the oddball; return 'gt', 'eq', or 'lt'."
    diff = sum(weight(b, oddball) for b in L) - sum(weight(b, oddball) for b in R)
    return ('gt' if diff > 0 else
            'lt' if diff < 0 else
            'eq')

def weight(ball, oddball) -> int: 
    return 101 if +ball == oddball else 99 if -ball == oddball else 100
    
def solve(puzzle) -> Tree or None:
    "Return a valid tree; one that solves the puzzle, or None."
    tree = find_tree(puzzle, puzzle.oddballs, puzzle.weighings)
    return tree if valid(tree, puzzle) else None
    
def follow(tree, oddball) -> Oddball:
    "Follow a path through the tree and return the oddball that the tree leads us to."
    if isinstance(tree, Oddball):
        return tree
    else:
        result = weigh(tree.L, tree.R, oddball)
        return follow(getattr(tree, result), oddball)
    
def valid(tree, puzzle) -> bool:
    "Does the strategy tree solve the puzzle correctly for all possible oddballs?"
    return (tree is not None and
            depth(tree) <= puzzle.weighings and 
            all(follow(tree, oddball) == oddball 
                for oddball in puzzle.oddballs))

def depth(tree) -> int:
    "Maximum depth of a strategy tree."
    return (0 if isinstance(tree, Oddball) else 
            1 + max(depth(tree.gt), depth(tree.eq), depth(tree.lt)))

Let's try out some of these functions:

In [2]:
p8 = Puzzle(8) 

p8.weighings, p8.balls, p8.oddballs

(3,
 [1, 2, 3, 4, 5, 6, 7, 8],
 {-8, -7, -6, -5, -4, -3, -2, -1, 1, 2, 3, 4, 5, 6, 7, 8})

In [3]:
# If we weigh balls 1, 2 against 3, 4, and the oddball is that 5 is lighter, the result should be 'eq'
weigh([1, 2], [3, 4], -5)

'eq'

In [4]:
weight(1, oddball=-5)

100

In [5]:
weight(5, oddball=-5)

99

# Strategy for Finding a Valid Tree

Now for the tricky part. We want to find a valid tree to solve a puzzle. The key idea is that a **weighing** gives us information by making a **partition** of the possible **oddballs** into entries for eaach of the three possible weighing results: `gt`, `eq`, or `lt`. Subsequent subtrees can handle each of the partitions.


In [6]:
def partition(L, R, oddballs) -> dict:
    "Build a dict of the possible outcomes (oddballs) from weighing L versus R."
    part = dict(gt=set(), eq=set(), lt=set())
    for odd in oddballs:
        part[weigh(L, R, odd)].add(odd)
    return part

For example, with 12 balls, if we weigh balls 1 and 2 on the left versus 11 and 12 on the right, then there are 4 ways  the left side can be greater than the right: either 1 or 2 is heavier or 11 or 12 is lighter. Similarly there are 4 ways of getting a `lt` weighing result. The remaining 16 possible oddballs&mdash;balls 3 through 10 being either heavier or lighter&mdash;show up in the `eq` entry of the partition:

In [7]:
p12 = Puzzle(12, 3)

partition([1, 2], [11, 12], p12.oddballs)

{'gt': {-12, -11, 1, 2},
 'eq': {-10, -9, -8, -7, -6, -5, -4, -3, 3, 4, 5, 6, 7, 8, 9, 10},
 'lt': {-2, -1, 11, 12}}

If this was the first weighing in our strategy tree, could we go on to solve the puzzle in 3 weighings? **No!** Any one weighing can at best partition the remaining possibilities into 3 equal entries. To solve the puzzle, we need every path in the tree to end up with only one possibility. So any two weighings can handle at most  3 &times; 3 = 9 possibilities; here we have 16, which is too many. We call this a **bad partition**.

The following is a **good partition** because each of the entries has 8 possibilities, and 8 is less than 9. (Note: being a good partition does not guarantee that the problem is solvable from there; but being a bad partition guarantees that it is not solvable.)

In [8]:
partition([1, 2, 3, 4], [9, 10, 11, 12], p12.oddballs)

{'gt': {-12, -11, -10, -9, 1, 2, 3, 4},
 'eq': {-8, -7, -6, -5, 5, 6, 7, 8},
 'lt': {-4, -3, -2, -1, 9, 10, 11, 12}}

So now we have a viable approach to implementing `find_tree`:

   - We call `find_tree(puzzle, oddballs, weighings)`. At the top level, the oddballs and number of weighings come from the puzzle. At recursive levels, we will reduce the number of oddball possibilities according to the partition, and the number of remaining weighings by 1 each time.
   - At each step we will randomly select two groups of balls, `L` and `R`, to be weighed.
   - We will then see what partition `L` and `R` gives us, and whether the partition is good or bad.
   - If the partition is bad, try another random selection of `L` and `R`.
   - Use a **greedy** approach where we accept the first good partition.
      (If we don't find a good partition after 1,000 tries, we give up.)
   - Once we have a good partition, we recursively find a tree for each of the branches of the partition.

In [9]:
def find_tree(puzzle, oddballs, weighings) -> Tree or Oddball or None:
    "Find a strategy tree that covers all the oddballs in the given number of weighings."
    if len(oddballs) == 1:
        return oddballs.pop() # One oddball possibility left; we're done: leaf node
    elif len(oddballs) == 0:
        return 0              # No oddball
    elif weighings == 0:
        return None           # Can't find a solution in the allowable weighings
    else:
        L, R, part = find_good_partition(puzzle, oddballs, weighings - 1)
        if not part: 
            return None
        subtrees = {r: find_tree(puzzle, part[r], weighings - 1) for r in part}
        if None not in subtrees.values(): 
            return Tree(L, R, **subtrees)
    
def find_good_partition(puzzle, oddballs, weighings) -> (list, list, dict):
    "Return (L, R, partition) such that no partition entry has more than 3**weighings oddballs."
    for _ in range(1000): 
        L, R = random_LR(puzzle, oddballs)
        part = partition(L, R, oddballs)
        if all(len(entry) <= 3 ** weighings for entry in part.values()):
            return L, R, part
    return [], [], {} # Fail
    
def random_LR(puzzle, oddballs):
    "Random choice of balls for L and R side."
    # Pick a random number of balls, B, then randomly pick B balls for each side.
    B = random.choice(range(1, (len(puzzle.balls) - 1) // 3 + 2))
    random.shuffle(puzzle.balls) 
    return puzzle.balls[:B], puzzle.balls[-B:]

Here we see that `find_good_partition` does its job:

In [10]:
find_good_partition(p12, p12.oddballs, 2)

([12, 3, 6, 9],
 [10, 5, 11, 7],
 {'gt': {-11, -10, -7, -5, 3, 6, 9, 12},
  'eq': {-8, -4, -2, -1, 1, 2, 4, 8},
  'lt': {-12, -9, -6, -3, 5, 7, 10, 11}})

It uses `random`, so it won't get the same result every time:

In [11]:
find_good_partition(p12, p12.oddballs, 2)

([2, 3, 1, 12],
 [11, 8, 9, 4],
 {'gt': {-11, -9, -8, -4, 1, 2, 3, 12},
  'eq': {-10, -7, -6, -5, 5, 6, 7, 10},
  'lt': {-12, -3, -2, -1, 4, 8, 9, 11}})

# Solving Some Puzzles

Now we're ready to solve puzzles!

In [12]:
solve(p12)

Tree(L=[2, 7, 5, 10], R=[4, 9, 11, 8], gt=Tree(L=[5, 9, 7, 8], R=[1, 4, 2, 3], gt=Tree(L=[8, 10, 5], R=[1, 2, 7], gt=5, eq=-4, lt=7), eq=Tree(L=[8, 3, 9, 5], R=[10, 2, 6, 11], gt=-11, eq=0, lt=10), lt=Tree(L=[8, 6, 5], R=[9, 1, 11], gt=-9, eq=2, lt=-8)), eq=Tree(L=[12, 2, 10, 9], R=[7, 6, 11, 3], gt=Tree(L=[8, 3, 12], R=[2, 11, 4], gt=12, eq=-6, lt=-3), eq=Tree(L=[6, 3, 11], R=[8, 1, 4], gt=-1, eq=0, lt=1), lt=Tree(L=[1, 7, 10], R=[11, 6, 12], gt=-12, eq=3, lt=6)), lt=Tree(L=[7, 6, 3, 1], R=[9, 8, 2, 5], gt=Tree(L=[6, 9, 12], R=[2, 8, 4], gt=-2, eq=-5, lt=0), eq=Tree(L=[4], R=[11], gt=4, eq=-10, lt=11), lt=Tree(L=[8, 2, 7], R=[4, 12, 1], gt=8, eq=9, lt=-7)))

OK, that's hard to read&mdash;my bad. Let's look at a much easier puzzle: 3 balls, 1 weighing allowed, and the odd ball can only be lighter:

In [13]:
tree = solve(Puzzle(3, 1, {-1}))
tree

Tree(L=[1], R=[2], gt=-2, eq=-3, lt=-1)

This tree says you weigh one ball against another (leaving the third unweighed), and the three possible weighing results tell you which of the three balls  is lighter. 



# Prettier Output

Let's make the output easier to read. I'll use `[1·2·3·4 ⟘ 9·10·11·12] ➔` to mean *"Weigh balls 1,2,3,4 versus 9,10,11,12 to get a result..."*. I'll indent each interior node in the tree, and I'll use `>:` to mean *the result when the left hand side is greater than the right in weight is...*  

Also, note that at the top node of a tree, there's no sense randomly shuffling the balls&mdash;the only choice that matters is how many balls, `B`, to put on each side. Putting `1·2·3·4` on the left is no different than `3·7·9·12`, because each ball is undifferentiated at the start. I'll alter `random_LR` for this.

In [14]:
def do(puzzle):
    "Print the solution to the puzzle as indented text."
    print(indented(solve(puzzle)))
    
def indented(tree, i=0, prefix=''):
    "Pretty, indented string of a strategy tree."
    if isinstance(tree, Tree):
        subtrees = (indented(tree.gt, i+1, '>:'), indented(tree.eq, i+1, '=:'), indented(tree.lt, i+1, '<:'))
        indent   = ('' if i == 0 else ('\n' + ' ' * 5 * i))
        return f'{indent}{prefix}[{items(tree.L)} ⟘ {items(tree.R)}] ➔ {" ".join(subtrees)})'
    elif tree == 0 or tree == None:
        return f'{prefix}{tree}'
    else:
        return f'{prefix}{tree:+d}'
    
def items(collection): return '·'.join(map(str, sorted(collection)))

def random_LR(puzzle, oddballs):
    "Random choice of balls for L and R side."
    # Pick a random number of balls, B, then pick B balls for each side.
    B = random.choice(range(1, (len(puzzle.balls) - 1) // 3 + 2))
    if oddballs == puzzle.oddballs:
        puzzle.balls.sort()
    else:
        random.shuffle(puzzle.balls) 
    return puzzle.balls[:B], puzzle.balls[-B:]

In [15]:
# 3 balls, 1 weighing, only lighter balls possible
do(Puzzle(3, 1, {-1}))

[1 ⟘ 3] ➔ >:-3 =:-2 <:-1)


In [16]:
# 3 balls, 2 weighings, lighter or heavier balls possible
do(Puzzle(3, 2))

[1 ⟘ 3] ➔ 
     >:[3 ⟘ 2] ➔ >:0 =:+1 <:-3) 
     =:[2 ⟘ 1] ➔ >:+2 =:0 <:-2) 
     <:[2 ⟘ 3] ➔ >:0 =:-1 <:+3))


In [17]:
# The original puzzle with 12 balls
do(p12)

[1·2·3·4 ⟘ 9·10·11·12] ➔ 
     >:[1·3·6·9 ⟘ 2·4·8·11] ➔ 
          >:[3·6·11·12 ⟘ 2·8·9·10] ➔ >:+3 =:+1 <:-11) 
          =:[12 ⟘ 1] ➔ >:0 =:-10 <:-12) 
          <:[1·7·11·12 ⟘ 4·5·8·9] ➔ >:-9 =:+2 <:+4)) 
     =:[3·8·10·11 ⟘ 4·6·7·9] ➔ 
          >:[4·9·10·12 ⟘ 1·6·8·11] ➔ >:-6 =:-7 <:+8) 
          =:[2·5 ⟘ 1·6] ➔ >:+5 =:0 <:-5) 
          <:[7·10·11 ⟘ 1·6·12] ➔ >:+7 =:-8 <:+6)) 
     <:[2·4·5·11 ⟘ 3·7·8·10] ➔ 
          >:[3·9 ⟘ 2·8] ➔ >:0 =:+11 <:-3) 
          =:[6·9·11 ⟘ 3·5·12] ➔ >:+9 =:-1 <:+12) 
          <:[5·7·11·12 ⟘ 2·6·9·10] ➔ >:-2 =:-4 <:+10)))


# Other Puzzles



We can do 12 balls in 3 weighings even if it is possible that all the balls weigh the same (we use `0` to denote this situation):

In [18]:
do(Puzzle(12, 3, {-1, 0, +1}))

[1·2·3·4 ⟘ 9·10·11·12] ➔ 
     >:[3·5·7·12 ⟘ 1·4·6·10] ➔ 
          >:[10 ⟘ 7] ➔ >:0 =:+3 <:-10) 
          =:[2·10·11 ⟘ 4·6·7] ➔ >:+2 =:-9 <:-11) 
          <:[4·7·11·12 ⟘ 5·6·8·9] ➔ >:+4 =:+1 <:-12)) 
     =:[2·6·8·12 ⟘ 1·7·10·11] ➔ 
          >:[2·8·12 ⟘ 1·6·11] ➔ >:+8 =:-7 <:+6) 
          =:[5·8·10·12 ⟘ 3·6·7·9] ➔ >:+5 =:0 <:-5) 
          <:[2·3·4 ⟘ 7·8·10] ➔ >:-8 =:-6 <:+7)) 
     <:[2·4·5·11 ⟘ 1·3·6·8] ➔ 
          >:[4·7·10·12 ⟘ 3·5·9·11] ➔ >:-3 =:-1 <:+11) 
          =:[3·11·12 ⟘ 5·7·10] ➔ >:+12 =:+9 <:+10) 
          <:[5·6·9 ⟘ 4·8·12] ➔ >:-4 =:-2 <:0)))


We can tackle much larger puzzles. With 4 weighings, we can theoretically handle up to 3<sup>4</sup> = 81 possibilities. Can we solve for 40 balls, each of which might be heavier or lighter?

In [19]:
do(Puzzle(40, 4))

None


Unfortunately, no. How about 39 balls, and how about we allow for the possibility that no ball is odd. That's 39 &times; 2 + 1 = 79 possibilities.

In [20]:
do(Puzzle(39, 4, {-1, 0, +1}))

[1·2·3·4·5·6·7·8·9·10·11·12·13 ⟘ 27·28·29·30·31·32·33·34·35·36·37·38·39] ➔ 
     >:[4·6·7·9·10·13·17·18·32·33·35·37·39 ⟘ 1·2·3·11·14·16·19·20·22·23·28·31·36] ➔ 
          >:[1·2·4·7·9·16·21·26·28·36·37·39 ⟘ 10·12·15·18·19·22·23·24·27·29·32·34] ➔ 
               >:[2·4·6·10·12·15·17·19·20·21·23·31·36 ⟘ 3·5·8·9·11·13·16·18·26·33·35·37·39] ➔ >:+4 =:+7 <:+9) 
               =:[10·12·18·19·23·25·30·36·39 ⟘ 4·6·8·14·15·24·28·31·35] ➔ >:-31 =:+13 <:+6) 
               <:[6·8·10·16·20·21·26·28·29·33·34 ⟘ 2·3·7·9·11·14·22·25·31·32·39] ➔ >:+10 =:-36 <:-28)) 
          =:[1·4·6·8·11·18·19·22·38 ⟘ 9·12·24·27·30·31·33·37·39] ➔ 
               >:[6·7·9·13·30·36 ⟘ 23·24·25·26·27·34] ➔ >:-27 =:+8 <:-30) 
               =:[1·6·17·19·26·30·31·35·37·39 ⟘ 5·9·14·15·18·20·23·24·27·34] ➔ >:-34 =:-29 <:+5) 
               <:[10·11·12·16·23·25·38 ⟘ 2·7·13·21·22·34·36] ➔ >:+12 =:0 <:-38)) 
          <:[5·6·8·9·11·13·19·21·22·31·34·36·37 ⟘ 1·2·10·12·14·15·17·24·27·30·32·33·38] ➔ 
               >:[2·4·18·26·38 

We can do **27** balls in 3 weighings if we know that the odd ball can only be lighter, not heavier. And we can do 26 balls under the condition that either an odd ball is lighter or all the balls weigh the same. In both cases there are 27 possibilities, so that's the most we can possibly do.

In [21]:
do(Puzzle(27, 3, {-1}))

[1·2·3·4·5·6·7·8·9 ⟘ 19·20·21·22·23·24·25·26·27] ➔ 
     >:[6·7·12·15·17·19·20·25 ⟘ 2·5·10·13·16·24·26·27] ➔ 
          >:[1·2·5·6·10·14·20·25·26 ⟘ 3·9·13·15·16·18·19·23·27] ➔ >:-27 =:-24 <:-26) 
          =:[6·11·14·16·22 ⟘ 1·15·21·25·27] ➔ >:-21 =:-23 <:-22) 
          <:[1·3·6·11·13·20·22·26 ⟘ 4·7·10·12·14·15·19·24] ➔ >:-19 =:-25 <:-20)) 
     =:[2·4·12·15·18·25 ⟘ 1·9·13·14·17·26] ➔ 
          >:[11·13·15·16·21·23 ⟘ 7·9·12·14·24·27] ➔ >:-14 =:-17 <:-13) 
          =:[5·7·12·16·17·22·23·25 ⟘ 6·8·9·10·13·19·24·26] ➔ >:-10 =:-11 <:-16) 
          <:[2·6·9·15·21 ⟘ 4·8·11·12·23] ➔ >:-12 =:-18 <:-15)) 
     <:[2·4·6·10·11·12·14·18·19 ⟘ 1·3·9·13·22·24·25·26·27] ➔ 
          >:[3·8·13·16·20·24·27 ⟘ 4·6·7·9·12·14·15] ➔ >:-9 =:-1 <:-3) 
          =:[2·3·8·9·12·15·19·22·23 ⟘ 1·5·6·10·14·17·25·26·27] ➔ >:-5 =:-7 <:-8) 
          <:[4·5·7·8·12·13·16·22 ⟘ 3·6·9·11·17·18·24·25] ➔ >:-6 =:-2 <:-4)))


In [22]:
do(Puzzle(26, 3, {-1, 0}))

[1·2·3·4·5·6·7·8·9 ⟘ 18·19·20·21·22·23·24·25·26] ➔ 
     >:[1·5·7·8·9·12·20·21·26 ⟘ 2·3·10·13·14·17·19·23·25] ➔ 
          >:[4·6·9·11·17·19·22·26 ⟘ 2·5·10·16·20·21·24·25] ➔ >:-25 =:-23 <:-19) 
          =:[1·15·22·25·26 ⟘ 7·14·18·19·20] ➔ >:-18 =:-24 <:-22) 
          <:[5·8·12·16·21·22·23·24 ⟘ 2·3·6·11·14·19·20·25] ➔ >:-20 =:-26 <:-21)) 
     =:[6·7·10·15·16·20·21·23 ⟘ 8·9·11·13·14·19·24·26] ➔ 
          >:[4·10·12·14·19·20 ⟘ 8·9·11·15·22·26] ➔ >:-11 =:-13 <:-14) 
          =:[6·11·12·14·21·22·23·25 ⟘ 7·8·9·13·15·16·17·20] ➔ >:-17 =:0 <:-12) 
          <:[5·11·12·16·19·24·25·26 ⟘ 2·6·8·10·17·18·20·23] ➔ >:-10 =:-15 <:-16)) 
     <:[5·8·9·12·13·14·19·23·25 ⟘ 1·2·6·16·17·18·21·22·24] ➔ 
          >:[2·7·8·9·10·13·20·21·22 ⟘ 5·6·11·15·16·17·19·25·26] ➔ >:-6 =:-1 <:-2) 
          =:[4·13·14·23·25 ⟘ 1·3·12·18·22] ➔ >:-3 =:-7 <:-4) 
          <:[2·3·8·13·14·15·22 ⟘ 1·6·7·9·18·20·26] ➔ >:-9 =:-5 <:-8)))


Can we solve the 13-ball, heavier-or-lighter problem, which has 26 possibilities? 

In [23]:
do(Puzzle(13))

None


**No**, and the reason is that there is no first weighing that partitions the 26 possibilities into 9/9/8; the best we can do is partition into 8/10/8 or 10/6/10:

In [24]:
partition([1, 2, 3, 4], [10, 11, 12, 13], Puzzle(13).oddballs)

{'gt': {-13, -12, -11, -10, 1, 2, 3, 4},
 'eq': {-9, -8, -7, -6, -5, 5, 6, 7, 8, 9},
 'lt': {-4, -3, -2, -1, 10, 11, 12, 13}}

In [25]:
partition([1, 2, 3, 4, 5], [9, 10, 11, 12, 13], Puzzle(13).oddballs)

{'gt': {-13, -12, -11, -10, -9, 1, 2, 3, 4, 5},
 'eq': {-8, -7, -6, 6, 7, 8},
 'lt': {-5, -4, -3, -2, -1, 9, 10, 11, 12, 13}}

Here's a puzzle with 25 balls, and the possibilities are that either one of the odd-numbered balls is heavier, or one of the even-numbered balls is lighter, or all the balls weigh the same.

In [26]:
p = Puzzle(25, 3)
p.oddballs = {(+b if b % 2 else -b) for b in p.balls} | {0}

do(p)

[1·2·3·4·5·6·7·8·9 ⟘ 17·18·19·20·21·22·23·24·25] ➔ 
     >:[1·6·7·9·10·16·18·19·24 ⟘ 2·3·4·8·11·15·17·23·25] ➔ 
          >:[8·9·13·15·20 ⟘ 2·7·12·14·21] ➔ >:+9 =:+1 <:+7) 
          =:[5·11·14·16·18·21·22·24 ⟘ 1·2·6·7·9·10·17·19] ➔ >:+5 =:-20 <:-22) 
          <:[8·13·16·17·19·23·25 ⟘ 2·3·4·9·11·15·24] ➔ >:-24 =:-18 <:+3)) 
     =:[1·7·11·16·19 ⟘ 4·12·13·15·23] ➔ 
          >:[1·3·6·8·16·22·25 ⟘ 2·5·12·17·20·21·24] ➔ >:-12 =:+11 <:0) 
          =:[12·14·19 ⟘ 3·10·21] ➔ >:-10 =:0 <:-14) 
          <:[3·4·5·7·8·10·15·24 ⟘ 9·13·17·18·19·20·22·25] ➔ >:+15 =:-16 <:+13)) 
     <:[2·7·14·16·17·21·22 ⟘ 3·6·15·18·19·24·25] ➔ 
          >:[7·9·11·22·23·25 ⟘ 6·8·10·12·18·21] ➔ >:-6 =:+17 <:+21) 
          =:[8·11·13·16·19·24 ⟘ 2·3·4·5·15·25] ➔ >:-4 =:+23 <:-8) 
          <:[2·8·9·10·11·12·13·25 ⟘ 4·6·14·15·17·20·23·24] ➔ >:+25 =:+19 <:-2)))


In [27]:
print(p.oddballs)

{0, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, -24, -22, -20, -18, -16, -14, -12, -10, -8, -6, -4, -2}


# What's Next?

- What other puzzles can you solve?
- Can you make a table of solvable and unsolvable puzzles?
- What happens when it is a possibility that *two or more* balls are odd?
- Can you prove which puzzles are unsolvable? Can you modify the `find_tree` function so that it only fails when the puzzle really is unsolvable, and not when it happens to have bad luck with `random`?
- What else can you discover?
